In [1]:
import sys
import csv
import os
# setting path
sys.path.append('../')

from utils.display import display_pixel_arry

from dcm_anonymizers.img_anonymizers import DCMImageAnonymizer
# from dcm_anonymizers.phi_detectors import DcmRobustPHIDetector

from pydicom import dcmread

from pathlib import Path

In [2]:
def get_subdirs_with_files(base_path):
    subdirs_with_files = []
    
    # Walk through each directory in the specified path
    for root, dirs, files in os.walk(base_path):
        if files:  # Check if the current directory contains any files
            subdirs_with_files.append(root)
    
    return subdirs_with_files

In [3]:
output_patient_id = "Pseudo-PHI-015"
series_idx = 0
dcm_filename = "00000105.dcm"

In [4]:
validation_data_path = "/home/r079a/Desktop/de-identification/dataset/midi-validation-data"

patient_mapping_path = Path(validation_data_path, 'output_data/mappings/patient_id_mapping.csv')

patient_mapping = {}
with open(patient_mapping_path, mode='r') as infile:
    reader = csv.reader(infile)
    next(reader, None)  # skip the headers
    patient_mapping = {rows[1]:rows[0] for rows in reader}

input_data_root = Path(validation_data_path, 'input_data')

target_patient_path = input_data_root / patient_mapping[output_patient_id]

all_series_paths = get_subdirs_with_files(target_patient_path)

In [5]:

sample_dicom = os.path.join(all_series_paths[series_idx], dcm_filename)

# Following one is important to traverse sequences
# sample_dicom = Path(validation_data_path, 'input_data/8571931193/1.4.860.1.0.6050381.6.464.2117248485162392317/1.4.860.1.0.6050381.6.464.1304159448982591337/00000001.dcm')


# sample_dicom = Path(validation_data_path, 'output_data/data/Pseudo-PHI-002/1.2.826.0.1.3680043.8.498.10746498679825505398104660274302695275/1.2.826.0.1.3680043.8.498.88870736155561815812592063497284637934/00000105.dcm')

# sample_dicom = Path(validation_data_path, 'input_data/2951928526/2.1.846.0.0.0624952.3.426.1171423858789191618/2.1.846.0.0.0624952.3.426.4666769529569841014/00000001.dcm')

# sample_dicom = "/home/r079a/Downloads/input_data/input_data/8172202354/3.4.823.0.1.5139398.5.009.3225903761765232034/3.4.823.0.1.5139398.5.009.2213188170240812441/00000105.dcm"

print(sample_dicom)

/home/r079a/Desktop/de-identification/dataset/midi-validation-data/input_data/4383555268/2.2.167.1.1.2390214.1.351.2355302027984409457/2.2.167.1.1.2390214.1.351.1379671703172623201/00000105.dcm


In [6]:
dataset = dcmread(sample_dicom)

In [7]:
pixel_data = dataset.get("pixel_array")

In [8]:
print("PixelData" in dataset)

True


In [9]:
print(dataset)

Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 176
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: MR Image Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 2.2.167.1.1.2390214.1.351.2957790087590339429
(0002, 0010) Transfer Syntax UID                 UI: Explicit VR Little Endian
(0002, 0012) Implementation Class UID            UI: 1.2.40.0.13.1.1
(0002, 0013) Implementation Version Name         SH: 'dcm4che-1.4.27'
-------------------------------------------------
(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'M_IR', 'M', 'IR']
(0008, 0012) Instance Creation Date              DA: '20110420'
(0008, 0013) Instance Creation Time              TM: '093403'
(0008, 0014) Instance Creator UID                UI: 2.2.167.1.1.2390214.1.351.3705126474999759704
(000

In [10]:
target_tag = (0x0020, 0x0011)
elem = dataset.get(target_tag)

elem.value = '1899617'

elem = dataset.get(target_tag)

elem

(0020, 0011) Series Number                       IS: '1899617'

In [11]:
# print(pixel_data[0].shape)

In [12]:
# display_pixel_arry(pixel_data[0])

In [13]:
# detector = DcmRobustPHIDetector()

In [14]:
# imganonymizer = DCMImageAnonymizer(phi_detector=detector)

In [15]:
# updated = imganonymizer.anonymize_dicom_image_data(dataset)

In [16]:
# updated

In [17]:
import pydicom
from pydicom.dataset import Dataset, DataElement
from pydicom.sequence import Sequence

from dcm_anonymizers.private_tags_extractor import PrivateTagsExtractor

In [18]:
ptags_dict = PrivateTagsExtractor('../docs/TCIAPrivateTagKB-02-01-2024-formatted.csv')

In [19]:
def extract_private_groups_n_creators(dataset):
    creators = []
    groups = []
    for element in dataset:
        if element.VR == 'OW':
            continue
        if element.tag.is_private:
            groups.append(f"{element.tag.group:04x}")
            if element.name == 'Private Creator' and element.value not in creators:                
                creators.append(element.value)

    groups = list(set(groups))

    return groups, creators

In [20]:
# groups, creators = extract_private_groups_n_creators(dataset)

# print(groups)
# print(creators)

In [21]:
# def create_pattents_from_element(element, parent_elements: list[DataElement]):
#     patterns = []
#     if len(parent_elements) == 0:
#         patterns = ptags_dict.search_patterns_from_element(element, creators)
#     else:
#         element_pattens = ptags_dict.search_patterns_from_element(element, creators)
#         for pttrn in element_pattens:
#             filtered_rows = ptags_dict.filter_by_pattern_n_vr(pttrn, element.VR)
#             print(pttrn, len(filtered_rows))                                                         
                                                              
#         for pe in parent_elements:
#             print(pe)

#     return patterns

# def traverse_dataset(dataset, parent_elements=[], is_root=True):
#     for elem in dataset:
#         tag = elem.tag
#         VR = elem.VR
#         value = elem.value
#         name = elem.name

#         # Check if the root element is private
#         if is_root:
#             if not elem.tag.is_private:
#                 continue
        
#         # Process the element
#         if isinstance(value, Sequence):
#             # If the value is a Sequence, recursively traverse each Dataset in the Sequence
#             # print(f"Sequence: {name} (Tag: {tag})")
#             updated_parent_elements = parent_elements.copy()
#             updated_parent_elements.append(elem)
#             for i, item in enumerate(value):
#                 # print(f"  Item {i} (Parent: {name})")
#                 traverse_dataset(item, parent_elements=updated_parent_elements, is_root=False)
#         else:
#             # process the data element
#             print(elem.tag)
#             patterns = create_pattents_from_element(elem, parent_elements)
#             print(patterns)
#             print("------------------")

In [22]:
# traverse_dataset(dataset)

### Test the Anonymizer on the Validation Data

In [23]:
from dcm_anonymizers.phi_detectors import DcmPHIDetector, DcmRobustPHIDetector
from dcm_anonymizers.tcia_deid import DCMTCIAAnonymizer
from dcm_anonymizers.private_tags_extractor import PrivateTagsExtractorV2

In [24]:
PRIVATE_TAGS_DICT = '../dcm_anonymizers/tcia_private_tags_dict.json'

# initialize model
phi_detector = DcmRobustPHIDetector(logging = True)
ptags_extr = PrivateTagsExtractorV2(
    private_tags_dict_path=PRIVATE_TAGS_DICT
)

In [25]:
anonymizer = DCMTCIAAnonymizer(
    phi_detector=None,
    notes_phi_detector=phi_detector,
    soft_detection=True,
    private_tags_extractor=ptags_extr,
    rules_json_path="../dcm_anonymizers/tcia_deid_attrs.json"
)

[2024-09-04 12:07:27,174] [   DEBUG] ps_3_3.py:97 - PS3.3 init
[2024-09-04 12:07:27,175] [   DEBUG] tcia_deid.py:67 - TCIA anonymizer init


In [26]:
anonymizer.anonymize_dataset(dataset)

In [30]:
phi_detector.missed_by_whitelist

{}

In [27]:
print(dataset)

Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 176
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: MR Image Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 1.2.826.0.1.3680043.8.498.93326537553902260764789771285200132251
(0002, 0010) Transfer Syntax UID                 UI: Explicit VR Little Endian
(0002, 0012) Implementation Class UID            UI: 1.2.40.0.13.1.1
(0002, 0013) Implementation Version Name         SH: 'dcm4che-1.4.27'
-------------------------------------------------
(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'M_IR', 'M', 'IR']
(0008, 0012) Instance Creation Date              DA: '20110818'
(0008, 0013) Instance Creation Time              TM: '093403'
(0008, 0014) Instance Creator UID                UI: 1.2.826.0.1.3680043.8.498.29956

In [28]:
anonymizer.history

{(0008, 0012): 'replace',
 (0008, 0020): 'replace',
 (0008, 0021): 'replace',
 (0008, 0022): 'replace',
 (0008, 0023): 'replace',
 (0040, 0244): 'replace',
 (0040, 0250): 'replace',
 (0008, 0014): 'replace_UID',
 (0008, 0018): 'replace_UID',
 (0020, 000d): 'replace_UID',
 (0020, 000e): 'replace_UID',
 (0020, 0052): 'replace_UID',
 (0008, 0050): 'empty',
 (0008, 0090): 'empty',
 (0010, 0030): 'empty',
 (0070, 0084): 'empty',
 (0010, 0010): 'empty',
 (0040, a075): 'empty',
 (0040, a123): 'empty',
 (0010, 4000): 'replace',
 (0040, 1400): 'replace',
 (0008, 1030): 'replace',
 (0008, 103e): 'replace',
 (0010, 21b0): 'replace',
 (0018, 1030): 'replace',
 (0018, 4000): 'replace',
 (0032, 1060): 'replace',
 (0117, 0010): 'tcia_keep',
 (0117, 10c4): 'tcia_delete'}

Evaluation of the anonymization process complete
=================================================
Anonymization Closeness Score 99.159%
=================================================
Mismatched Tags Summary:
--------------------------------------------
        Acquisition Device Processing Description: 1 -> 4
        Study Comments: 2 -> 4,16
        [Image actual date]: 1 -> 5
        [Product id]: 1 -> 6
        [Pulse Sequence Date]: 1 -> 6
        [Image from which prescribed]: 1 -> 6
        [Screen Format]: 1 -> 6
        [Vas collapse flag]: 1 -> 6
        [Scanner Table Entry (single gradient coil systems only)/Scanner Table Entry + Gradient Coil Selected]: 1 -> 6
        Patient State: 1 -> 10
        Allergies: 3 -> 11,15,17
        Reason for the Requested Procedure: 3 -> 11,15,17
        Requested Procedure Comments: 3 -> 11,15,17
        Reason for the Imaging Service Request: 3 -> 11,15,17
        Acquisition DateTime: 1 -> 12
        Study Description: 2 -> 13,14
        Code Meaning: 2 -> 13,14
        Protocol Name: 1 -> 13
        Date of Last Calibration: 2 -> 13,16
        Image Comments: 1 -> 13
        Pixel Aspect Ratio: 1 -> 16
        [GradientMode]: 1 -> 19
        Comments on the Performed Procedure Step: 1 -> 19
        Image Type: 1 -> 20
        Planar Configuration: 1 -> 20
        Private Creator: 1 -> 20
        Series Description: 1 -> 23
        Secondary Capture Device ID: 1 -> 24
        Overlay Rows: 1 -> 24
        Overlay Columns: 1 -> 24
        Number of Frames in Overlay: 1 -> 24
        Overlay Description: 1 -> 24
        Overlay Type: 1 -> 24
        Overlay Origin: 1 -> 24
        Image Frame Origin: 1 -> 24
        Overlay Bits Allocated: 1 -> 24
        Overlay Bit Position: 1 -> 24
---------------------------------------------